In [27]:
%load_ext autoreload
%autoreload 2
# wd = '/Users/marcinsawinski/Documents/GitHub/factue-task2'
wd = '/mnt/openfact/users/msawinski/factue-task2'
import sys, os
os.chdir(wd)
from factue.methods.llm_calls import make_call, load_template_parts, load_template_list
from factue.methods.llm_langchain.llm import Llm
from factue.utils.types import ModelMode, ModelProvider, ModelName, Job
from factue.utils.parsers import extract_json_from_payload
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
for x in load_template_list(job='persuasion', step="detect", prompt_version='v001').keys():
    print(f'"{x}"')

"Name_Calling-Labeling"
"Doubt"
"Questioning_the_Reputation"
"Appeal_to_Fear-Prejudice"
"Appeal_to_Values"
"Exaggeration-Minimisation"
"Loaded_Language"
"Repetition"
"new"


In [2]:
llm_params = {
            "model_name": ModelName.LLAMA_32_3B,
            "model_provider": ModelProvider.OLLAMA,
            "model_mode": ModelMode.CHAT,
            "seed": -1,
        }
llm = Llm(**llm_params)

seed:  -1


In [3]:
text = """Poseł Anna Gembicka:
Bardzo dziękuję.
Panie Marszałku! Wysoka Izbo! Pani Minister!
Przypomnę pani kilka faktów, bo widzę, że faktycznie w wielu obszarach jest pani zielona.
"""
df = pd.DataFrame({
    'txt': [
        text,
        text
    ]
})
df

,txt
0,Poseł Anna Gembicka:\nBardzo dziękuję.\nPanie ...
1,Poseł Anna Gembicka:\nBardzo dziękuję.\nPanie ...


In [21]:
schema = {
    "type": "object",
    "properties": {
        "Description": {"type": "string"},
        "Verdict": {"type": "boolean"}
    },
    "required": ["Description", "Verdict"],
    "additionalProperties": True
}

call_result = df.apply(lambda row: make_call(
                llm=llm,
                job=Job.PERSUASION.value,
                step='detect',
                prompt_id="attack/Name_Calling-Labeling_v001",
                variables={"text": row['txt']},
                max_iterations=2,
                json_schema=schema,
                max_retries=0)
            ,axis=1,)
call_result.tolist()

2025-05-06 18:19:24,407 [INFO] httpx: HTTP Request: POST http://0.0.0.0:5000/api/chat "HTTP/1.1 200 OK"
2025-05-06 18:19:25,327 [INFO] factue.methods.llm_calls: True
2025-05-06 18:19:25,408 [INFO] httpx: HTTP Request: POST http://0.0.0.0:5000/api/chat "HTTP/1.1 200 OK"
2025-05-06 18:19:26,375 [INFO] factue.methods.llm_calls: True
2025-05-06 18:19:26,431 [INFO] httpx: HTTP Request: POST http://0.0.0.0:5000/api/chat "HTTP/1.1 200 OK"
2025-05-06 18:19:27,385 [INFO] factue.methods.llm_calls: True
2025-05-06 18:19:27,457 [INFO] httpx: HTTP Request: POST http://0.0.0.0:5000/api/chat "HTTP/1.1 200 OK"
2025-05-06 18:19:29,576 [INFO] factue.methods.llm_calls: False


[[{'raw': '{"Description": "Technique used: Yes — Name Calling / Labeling is present. The use of titles (Panie Marszałku, Pani Minister) and honorifics (Wysoka Izbo) creates a formality and emphasizes the speaker\'s position. Additionally, the phrase \\"Pani zielona\\" is an example of Name Calling - Labeling: it labels Anna Gembicka with a pejorative term that refers to her political views or affiliations, without providing any evidence or argumentation.", "Verdict": true}',
   'extra_properties': None,
   'think_content': None,
   'extra_content': None,
   'illegal_value': {},
   'is_valid': True,
   'description': 'Technique used: Yes — Name Calling / Labeling is present. The use of titles (Panie Marszałku, Pani Minister) and honorifics (Wysoka Izbo) creates a formality and emphasizes the speaker\'s position. Additionally, the phrase "Pani zielona" is an example of Name Calling - Labeling: it labels Anna Gembicka with a pejorative term that refers to her political views or affiliati

In [13]:
from collections import Counter, defaultdict


In [25]:
from collections import defaultdict

def extract_all_fields_from_list_of_dicts(list_of_dicts):
    result = defaultdict(list)
    for d in list_of_dicts:
        if d is not None and d:
            result['think_content'].append(d.get('think_content', None))
            json_data = d.get('json_data')
            if isinstance(json_data, dict):
                for k, v in json_data.items():
                    result[k].append(v)
            else:
                result['error'].append("Invalid or missing json_data")
        else:
            result['error'].append("No content extracted")
    return dict(result)

extract_all_fields_from_list_of_dicts(call_result.tolist())

AttributeError: 'list' object has no attribute 'get'

In [ ]:
extract_all_fields_from_list_of_dicts

In [24]:
result = defaultdict(list)
for j in call_result.tolist():
    print("j",j)
    for k in j:
            print("k",k)
            # result[k].append(j[k])
    else:
        result['error'].append("No content appended")
result

j [{'raw': '{"Description": "Technique used: Yes — Name Calling / Labeling is present. The use of titles (Panie Marszałku, Pani Minister) and honorifics (Wysoka Izbo) creates a formality and emphasizes the speaker\'s position. Additionally, the phrase \\"Pani zielona\\" is an example of Name Calling - Labeling: it labels Anna Gembicka with a pejorative term that refers to her political views or affiliations, without providing any evidence or argumentation.", "Verdict": true}', 'extra_properties': None, 'think_content': None, 'extra_content': None, 'illegal_value': {}, 'is_valid': True, 'description': 'Technique used: Yes — Name Calling / Labeling is present. The use of titles (Panie Marszałku, Pani Minister) and honorifics (Wysoka Izbo) creates a formality and emphasizes the speaker\'s position. Additionally, the phrase "Pani zielona" is an example of Name Calling - Labeling: it labels Anna Gembicka with a pejorative term that refers to her political views or affiliations, without prov

defaultdict(list, {'error': ['No content appended', 'No content appended']})

In [9]:
extract_all_fields_from_list_of_jsons(result.tolist())

TypeError: unhashable type: 'dict'

In [29]:
def expand_series_of_dict_lists(series):
    # Extract all keys from all dicts to get full set of keys
    all_keys = set()
    for lst in series:#.dropna():
        for d in lst:
            all_keys.update(d.keys())

    # For each row, create a dict: key -> list of values for that key
    def extract_keys(dicts):
        return {key: [d.get(key) for d in dicts] for key in all_keys}

    return series.apply(extract_keys).apply(pd.Series)

# Apply the function
df_expanded = expand_series_of_dict_lists(call_result)
df_expanded

,illegal_value,description,extra_properties,extra_content,raw,think_content,is_valid,verdict
0,"[{}, {}]",[Technique used: Yes — Name Calling / Labeling...,"[None, None]","[None, None]","[{""Description"": ""Technique used: Yes — Name C...","[None, None]","[True, True]","[True, True]"
1,"[{}, {}]",[Technique used: Yes — Name Calling / Labeling...,"[None, None]","[None, None]","[{""Description"": ""Technique used: Yes — Name C...","[None, None]","[True, False]","[True, None]"
